# 03. Scenario analysis
## Project: Bicycle node network loop analysis

This notebook analyses loops created in notebook 01 with different scenarios.  
For now explore two scenarios:  
1. Family with small children: 5-20km, max slope 4%
1. Teenage/adult leisure cyclist: 10-40km, max slope 6%

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2024-10-07  
Last modified: 2024-10-07

## To do
- [ ] Refactor with functions for restrictions
- [ ] Sudden bad weather/accident scenario for family: if k km travelled, what is shortest path back with/without/how much overlaps?
- [ ] Re-think or drop due to coastline effects: Kommune cohesion: Loops vs distance to nearest kommune
- [ ] How much of network due to topography / existing paths?
- [ ] Study ideal triangle grid?
- [ ] Quantify?: information explosion huge first, but gets smaller with each node passed
- [X] POI analysis

## Parameters

In [ ]:
%run -i setup_parameters.py
load_data = True  # Set to False if data are huge and have already been loaded
debug = True  # Set to True for extra plots and verbosity

if PLOTLOGSCALE:
    logtextplot = "2^"
    logtextsave = "_logscale"
else:
    logtextplot = ""
    logtextsave = ""

## Functions

In [ ]:
%run -i functions.py

## Load data

This can take several minutes.

In [ ]:
if load_data:
    with open(
        PATH["data_out"] + "loopcensus_" + str(LOOP_NUMNODE_BOUND) + ".pkl", "rb"
    ) as f:
        allloops = pickle.load(f)
        alllooplengths = pickle.load(f)
        allloopnumnodes = pickle.load(f)
        allloopmaxslopes = pickle.load(f)
        G = pickle.load(f)
        LOOP_NUMNODE_BOUND = pickle.load(f)
        nodes_id = pickle.load(f)
        nodes_coords = pickle.load(f)
        numloops = pickle.load(f)
        faceloops = pickle.load(f)

In [ ]:
# Plot to double-check
if debug:
    plotCheck(
        G,
        nodes_id,
        nodes_coords,
        vertex_size=get_vertexsize(G.vcount()),
        edge_width=get_edgewidth(G.ecount()),
    )

## Analyse

### Max slopes

In [ ]:
linkmaxslopes = [e["max_slope"] for e in G.es]
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0.16, 0.16, 0.76, 0.75])

histxy = axes.hist(linkmaxslopes, bins=[i / 4 for i in list(range(32))], density=False)
axes.plot([MAXSLOPE_LIMIT, MAXSLOPE_LIMIT], [0, max(histxy[0])], ":r")
axes.text(
    MAXSLOPE_LIMIT * 0.95,
    max(histxy[0]),
    str(
        round(
            len([i for i, x in enumerate(linkmaxslopes) if (x < MAXSLOPE_LIMIT)])
            / len(linkmaxslopes)
            * 100
        )
    )
    + "%",
    horizontalalignment="right",
    verticalalignment="top",
)
axes.text(
    MAXSLOPE_LIMIT * 1.05,
    max(histxy[0]),
    str(
        round(
            len([i for i, x in enumerate(linkmaxslopes) if (x >= MAXSLOPE_LIMIT)])
            / len(linkmaxslopes)
            * 100
        )
    )
    + "%",
    horizontalalignment="left",
    verticalalignment="top",
    color="red",
)
axes.set_xlabel("Max slope [%]")
axes.set_ylabel("Frequency")
axes.set_title("Link max slopes")
axes.set_xlim([0, 8])

fig.savefig(PATH["plot"] + "maxslopes_maxslope" + str(MAXSLOPE_LIMIT))

### Restrict to scenario lengths

In [ ]:
allloops_bound = {}
for sourcenode in tqdm(allloops, desc="Restrict to scenario lenghts"):
    this_lengths = np.array(allloops[sourcenode]["lengths"]) * MPERUNIT
    mask_this = (this_lengths >= RESTRICTIONS["looplength_min"]) & (
        this_lengths <= RESTRICTIONS["looplength_max"]
    )
    allloops_bound[sourcenode] = {
        "loops": list(compress(allloops[sourcenode]["loops"], mask_this)),
        "lengths": list(compress(allloops[sourcenode]["lengths"], mask_this)),
        "numnodes": list(compress(allloops[sourcenode]["numnodes"], mask_this)),
        "water_profile": list(
            compress(allloops[sourcenode]["water_profile"], mask_this)
        ),
        "poi_diversity": list(
            compress(allloops[sourcenode]["poi_diversity"], mask_this)
        ),
    }

In [ ]:
vertex_sizes = []
for k in range(len(allloops_bound.keys())):
    try:
        if PLOTLOGSCALE:
            vertex_sizes.append(np.log2(len(allloops_bound[k]["loops"]) + 1))
        else:
            vertex_sizes.append(len(allloops_bound[k]["loops"]))
    except:
        vertex_sizes.append(0)

numloops_max = max(vertex_sizes)
vertex_sizes = [i / (numloops_max / 20) for i in vertex_sizes]

In [ ]:
fig = plotCheck(
    G,
    nodes_id,
    nodes_coords,
    vertex_size=vertex_sizes,
    edge_width=get_edgewidth(G.ecount()),
)
plt.text(0, 0.04, "Max # of loops: " + logtextplot + str(round(numloops_max, 2)))
plt.tight_layout()

fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(RESTRICTIONS["looplength_min"])
    + "to"
    + str(RESTRICTIONS["looplength_max"])
    + logtextsave
)

### Restrict to scenario gradients

In [ ]:
allloops_bound = {}
for sourcenode in tqdm(allloops, desc="Restrict to scenario gradients"):
    this_lengths = np.array(allloops[sourcenode]["lengths"]) * MPERUNIT
    this_maxslopes = np.array(allloops[sourcenode]["max_slopes"])
    mask_this = this_lengths >= RESTRICTIONS["looplength_min"]
    mask_this &= this_lengths <= RESTRICTIONS["looplength_max"]
    mask_this &= this_maxslopes <= RESTRICTIONS["slope_max"]
    allloops_bound[sourcenode] = {
        "loops": list(compress(allloops[sourcenode]["loops"], mask_this)),
        "lengths": list(compress(allloops[sourcenode]["lengths"], mask_this)),
        "numnodes": list(compress(allloops[sourcenode]["numnodes"], mask_this)),
        "water_profile": list(
            compress(allloops[sourcenode]["water_profile"], mask_this)
        ),
        "poi_diversity": list(
            compress(allloops[sourcenode]["poi_diversity"], mask_this)
        ),
    }

In [ ]:
vertex_sizes = []
for k in range(len(allloops_bound.keys())):
    try:
        if PLOTLOGSCALE:
            vertex_sizes.append(np.log2(len(allloops_bound[k]["loops"]) + 1.01))
        else:
            vertex_sizes.append(len(allloops_bound[k]["loops"]))
    except:
        vertex_sizes.append(0)

numloops_max = max(vertex_sizes)
vertex_sizes = [i / (numloops_max / 20) for i in vertex_sizes]

edge_colors = []
for e in G.es:
    if e["max_slope"] > RESTRICTIONS["slope_max"]:
        edge_colors.append("red")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(
    G,
    nodes_id,
    nodes_coords,
    vertex_size=vertex_sizes,
    edge_color=edge_colors,
    edge_width=get_edgewidth(G.ecount()),
)
plt.text(
    0,
    0.12,
    "Red: Gradient >" + str(RESTRICTIONS["slope_max"]) + "%",
    fontsize=8,
)
plt.text(0, 0.04, "Max # of loops: " + logtextplot + str(round(numloops_max, 2)))
plt.tight_layout()

fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(RESTRICTIONS["looplength_min"])
    + "to"
    + str(RESTRICTIONS["looplength_max"])
    + "_maxslope"
    + str(RESTRICTIONS["slope_max"])
    + logtextsave
)

### Restrict to water limits

In [ ]:
allloops_bound_watered = {}
for sourcenode in allloops_bound:
    numloops = len(allloops_bound[sourcenode]["loops"])
    mask_this = [True] * numloops
    for i in range(numloops):
        wp = allloops_bound[sourcenode]["water_profile"][i]
        water_enough = True
        if wp:  # There is water on the way somewhere. Check distances
            for w in wp:
                if w > RESTRICTIONS["waterlength_max"]:
                    water_enough = False
                    break
            if water_enough and (
                allloops_bound[sourcenode]["lengths"][i] - wp[-1]
                > RESTRICTIONS["waterlength_max"]
            ):
                water_enough = False
        else:  # No water on the way, so the loop is only valid if short enough
            if (
                allloops_bound[sourcenode]["lengths"][i]
                > RESTRICTIONS["waterlength_max"]
            ):
                water_enough = False
        mask_this[i] = water_enough
    allloops_bound_watered[sourcenode] = {
        "loops": list(compress(allloops_bound[sourcenode]["loops"], mask_this)),
        "lengths": list(compress(allloops_bound[sourcenode]["lengths"], mask_this)),
        "numnodes": list(compress(allloops_bound[sourcenode]["numnodes"], mask_this)),
        "water_profile": list(
            compress(allloops_bound[sourcenode]["water_profile"], mask_this)
        ),
        "poi_diversity": list(
            compress(allloops_bound[sourcenode]["poi_diversity"], mask_this)
        ),
    }

In [ ]:
vertex_sizes = []
for k in range(len(allloops_bound_watered.keys())):
    try:
        if PLOTLOGSCALE:
            vertex_sizes.append(np.log2(len(allloops_bound_watered[k]["loops"]) + 1.01))
        else:
            vertex_sizes.append(len(allloops_bound_watered[k]["loops"]))

    except:
        vertex_sizes.append(0)

numloops_max = max(vertex_sizes)
vertex_sizes = [i / (numloops_max / 20) for i in vertex_sizes]

edge_colors = []
for e in G.es:
    if e["max_slope"] > RESTRICTIONS["slope_max"]:  # and not e["has_water"]:
        edge_colors.append("red")
    # elif e["max_slope"] > RESTRICTIONS["slope_max"] and e["has_water"]:
    #     edge_colors.append("orange")
    elif e["max_slope"] <= RESTRICTIONS["slope_max"] and e["has_water"]:
        edge_colors.append("blue")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(
    G,
    nodes_id,
    nodes_coords,
    vertex_size=vertex_sizes,
    edge_color=edge_colors,
    edge_width=get_edgewidth(G.ecount()),
)
plt.text(
    0,
    0.17,
    "Blue: Gradient <=" + str(RESTRICTIONS["slope_max"]) + "% and water",
    fontsize=8,
)
# plt.text(0,0.17, "Orange: Gradient >"+ str(RESTRICTIONS["slope_max"])+"% and water", fontsize=8);
plt.text(
    0,
    0.12,
    "Red: Gradient >" + str(RESTRICTIONS["slope_max"]) + "%",
    fontsize=8,
)
plt.text(0, 0.04, "Max # of loops: " + logtextplot + str(round(numloops_max, 2)))
plt.tight_layout()

fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(RESTRICTIONS["looplength_min"])
    + "to"
    + str(RESTRICTIONS["looplength_max"])
    + "_maxslope"
    + str(RESTRICTIONS["slope_max"])
    + "_waterlength_max"
    + str(RESTRICTIONS["waterlength_max"])
    + logtextsave
)

### Restrict with POI diversity

In [ ]:
allloops_bound_watered_diverse = {}
for sourcenode in allloops_bound_watered:
    numloops = len(allloops_bound_watered[sourcenode]["loops"])
    mask_this = [False] * numloops
    for i in range(numloops):
        pd = allloops_bound_watered[sourcenode]["poi_diversity"][i]
        if pd >= 3:
            mask_this[i] = True
    allloops_bound_watered_diverse[sourcenode] = {
        "loops": list(compress(allloops_bound_watered[sourcenode]["loops"], mask_this)),
        "lengths": list(
            compress(allloops_bound_watered[sourcenode]["lengths"], mask_this)
        ),
        "numnodes": list(
            compress(allloops_bound_watered[sourcenode]["numnodes"], mask_this)
        ),
        "water_profile": list(
            compress(allloops_bound_watered[sourcenode]["water_profile"], mask_this)
        ),
        "poi_diversity": list(
            compress(allloops_bound_watered[sourcenode]["poi_diversity"], mask_this)
        ),
    }

In [ ]:
vertex_sizes = []
for k in range(len(allloops_bound_watered_diverse.keys())):
    try:
        if PLOTLOGSCALE:
            vertex_sizes.append(
                np.log2(len(allloops_bound_watered_diverse[k]["loops"]) + 1.01)
            )
        else:
            vertex_sizes.append(len(allloops_bound_watered_diverse[k]["loops"]))
    except:
        vertex_sizes.append(0)

numloops_max = max(vertex_sizes)
vertex_sizes = [i / (numloops_max / 20) for i in vertex_sizes]

edge_colors = []
for e in G.es:
    if e["max_slope"] > RESTRICTIONS["slope_max"]:  # and not e["has_water"]:
        edge_colors.append("red")
    # elif e["max_slope"] > RESTRICTIONS["slope_max"] and e["has_water"]:
    #     edge_colors.append("orange")
    elif (
        e["max_slope"] <= RESTRICTIONS["slope_max"]
        and e["has_water"]
        and e["poi_diversity"] >= 3
    ):
        edge_colors.append("blue")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(
    G,
    nodes_id,
    nodes_coords,
    vertex_size=vertex_sizes,
    edge_color=edge_colors,
    edge_width=get_edgewidth(G.ecount()),
)
plt.text(
    0,
    0.17,
    "Blue: Gradient <=" + str(RESTRICTIONS["slope_max"]) + "% and water and diverse",
    fontsize=8,
)
# plt.text(0,0.17, "Orange: Gradient >"+ str(RESTRICTIONS["slope_max"])+"% and water", fontsize=8);
plt.text(
    0,
    0.12,
    "Red: Gradient >" + str(RESTRICTIONS["slope_max"]) + "%",
    fontsize=8,
)
plt.text(0, 0.04, "Max # of loops: " + logtextplot + str(round(numloops_max, 2)))
plt.tight_layout()

fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(RESTRICTIONS["looplength_min"])
    + "to"
    + str(RESTRICTIONS["looplength_max"])
    + "_maxslope"
    + str(RESTRICTIONS["slope_max"])
    + "_waterlength_max"
    + str(RESTRICTIONS["waterlength_max"])
    + "_poidiversity3"
    + logtextsave
)